In [1]:
from games_setup import *
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common.reaction import Reaction
from SBMLLint.games.som import SOM
from SBMLLint.common.simple_sbml import SimpleSBML

import collections
import itertools
import networkx as nx
import numpy as np
import pandas as pd
import scipy
#
from SBMLLint.games.message import Message

Current Directory: /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [2]:
simple = load_file_from_curated_data(8)
for r in simple.reactions:
  #if r.category != cn.REACTION_BOUNDARY:
    print(r.makeIdentifier(is_include_kinetics=False))

reaction1:  -> C
reaction2: C -> 
reaction3: C -> 
reaction4:  -> M
reaction5: M -> 
reaction6:  -> X
reaction7: X -> 
reaction8: C + Y -> Z
reaction9: Z -> C + Y
reaction10: Z -> C
reaction11: Z -> Y
reaction12:  -> Y
reaction13: Y -> 


In [3]:
m = Message(simple)

In [4]:
m.reactions

[reaction8: C + Y -> Z; a1 * C * Y,
 reaction9: Z -> C + Y; a2 * Z,
 reaction10: Z -> C; alpha * d1 * Z,
 reaction11: Z -> Y; alpha * kd * Z]

In [5]:
m.molecules

['C', 'Z', 'Y']

In [6]:
m.stoichiometry_matrix

,reaction8,reaction9,reaction10,reaction11
C,-1.0,1.0,1.0,0.0
Z,1.0,-1.0,-1.0,-1.0
Y,-1.0,1.0,0.0,1.0


In [8]:
s = m.stoichiometry_matrix
a, b = m.decomposeMatrix(s.T)
print(a)
print(b)

[[ 1.  0.  0.]
 [-1.  1.  0.]
 [ 1.  0.  1.]]
              C    Z    Y
reaction8  -1.0  0.0  0.0
reaction9   1.0  0.0  0.0
reaction10  1.0  0.0 -1.0
reaction11  0.0 -1.0  1.0


In [10]:
s.T

,C,Z,Y
reaction8,-1.0,1.0,-1.0
reaction9,1.0,-1.0,1.0
reaction10,1.0,-1.0,0.0
reaction11,0.0,-1.0,1.0


In [21]:
mat_df = m.stoichiometry_matrix
mat_t = mat_df.T

mat_t
idx_mat_t = mat_t.index
# # LU decomposition
mat_lu = scipy.linalg.lu(mat_t)
# # inverse pivot matrix
p_inv = scipy.linalg.inv(mat_lu[0])
pivot_index = [list(k).index(1) for k in p_inv]
new_idx_mat_t = [idx_mat_t[idx] for idx in pivot_index]
# # row reduced matrix
row_reduced = pd.DataFrame(mat_lu[2], index=new_idx_mat_t, columns=mat_t.columns)
row_reduced

ValueError: Shape of passed values is (3, 3), indices imply (3, 4)

In [14]:
idx_mat_t

Index(['reaction8', 'reaction9', 'reaction10', 'reaction11'], dtype='object')

In [23]:
scipy.linalg.inv(mat_lu[0])

array([[ 1., -0.,  0.,  0.],
       [ 0., -0.,  0.,  1.],
       [ 0., -0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.]])

In [17]:
mat_lu[1]

array([[ 1.,  0.,  0.],
       [-0.,  1.,  0.],
       [-1., -0.,  1.],
       [-1., -0., -0.]])

In [18]:
mat_lu[2]

array([[-1.,  1., -1.],
       [ 0., -1.,  1.],
       [ 0.,  0., -1.]])

In [31]:
mat_df.T

,C,Z,Y
reaction8,-1.0,1.0,-1.0
reaction9,1.0,-1.0,1.0
reaction10,1.0,-1.0,0.0
reaction11,0.0,-1.0,1.0


In [35]:
from sympy import *
from sympy.matrices import *

In [52]:
# compare sympy with scipy decomposition => go with Sympy
mat_lu[1]

array([[ 1.,  0.,  0.],
       [-0.,  1.,  0.],
       [-1., -0.,  1.],
       [-1., -0., -0.]])

In [53]:
m = Matrix(mat_df.T)
m_lu = m.LUdecomposition()
print(m_lu[0])
print()
print(m_lu[1])
print()
m_lu[0]

Matrix([[1, 0, 0, 0], [0, 1, 0, 0], [-1.00000000000000, 0, 1, 0], [-1.00000000000000, 0, 0, 1]])

Matrix([[-1.00000000000000, 1.00000000000000, -1.00000000000000], [0, -1.00000000000000, 1.00000000000000], [0, 0, -1.00000000000000], [0, 0, 0]])



Matrix([
[   1, 0, 0, 0],
[   0, 1, 0, 0],
[-1.0, 0, 1, 0],
[-1.0, 0, 0, 1]])

In [33]:
M = Matrix([[-1, 1, -1, 1, 0, 0, 0],
          [1, -1, 1, 0, 1, 0, 0],
          [1, -1, 0, 0, 0, 1, 0],
          [0, -1, 1, 0, 0, 0, 1]])
M

Matrix([
[-1,  1, -1, 1, 0, 0, 0],
[ 1, -1,  1, 0, 1, 0, 0],
[ 1, -1,  0, 0, 0, 1, 0],
[ 0, -1,  1, 0, 0, 0, 1]])

In [34]:
M.rref()

(Matrix([
 [1, 0, 0, 0, 1,  0, -1],
 [0, 1, 0, 0, 1, -1, -1],
 [0, 0, 1, 0, 1, -1,  0],
 [0, 0, 0, 1, 1,  0,  0]]), (0, 1, 2, 3))